<a href="https://colab.research.google.com/github/safoura-banihashemi/Augmentation_Legal_Texts/blob/main/Augmentation_NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from collections import Counter
from itertools import chain
import ast
import json

## Exploring Dataset

In [ ]:
# Load dataset
data = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Augmentation/demosthenes.pkl")
data.head(10)

,Document,Split,Name,Id,Text,Type,Scheme,Start
0,A2008_Commission of the European Communities v...,1,prem,A1,"It must be held, first, that while Article 4(c...",L,Rule,18211
1,A2008_Commission of the European Communities v...,1,prem,A2,"Also, it is clear from consistent case-law tha...",L,"[Itpr, Prec, Rule]",18738
2,A2008_Commission of the European Communities v...,1,prem,A3,The Court deduces from this that Article 67 CS...,L,"[Prec, Rule]",19176
3,A2008_Commission of the European Communities v...,1,prem,A4,The Court has also held that action taken unde...,L,"[Prec, Rule]",19558
4,A2008_Commission of the European Communities v...,1,prem,A5,"The Court has held, in particular, that it is ...",L,"[Itpr, Prec, Rule]",19849
5,A2008_Commission of the European Communities v...,1,prem,A6,The Court has also held that the first indent ...,L,"[Prec, Rule]",20364
6,A2008_Commission of the European Communities v...,1,prem,A7,The Court accordingly held that a preferential...,L,"[Prec, Rule]",20757
7,A2008_Commission of the European Communities v...,1,prem,A8,"Finally, State aid granted to an undertaking f...",L,Itpr,21051
8,A2008_Commission of the European Communities v...,1,prem,A9,"In the light of the foregoing, it must consequ...",L,"[Itpr, Rule]",21327
9,A2008_Commission of the European Communities v...,1,prem,A10,"Contrary to what the German Government claims,...","[F, L]","[Itpr, Rule]",21545


In [ ]:
data.sample(10)

,Document,Split,Name,Id,Text,Type,Scheme,Start
623,A2017_European Commission v TV2_Danmark A_S,5,prem,A8,"Therefore, even if the sums corresponding to t...",L,Prec,20506
1970,R2016_Netherlands Maritime Technology Associat...,4,prem,B3,"The General Court observed, however, that sinc...",F,NaN,28515
913,A2018_Scuola Elementare Maria Montessori Srl v...,3,prem,D30,"As noted in paragraph 93 above, the General Co...",F,NaN,54210
255,A2011_European Commission (C-106_09 P) and Kin...,2,prem,B26,"In that regard, it should be observed that the...",F,NaN,54927
1914,R2016_Hellenic Republic v European Commission,4,prem,B5,"Consequently, that argument must be rejected a...",F,NaN,31479
1170,R2004_Italian Republic v Commission of the Eur...,1,prem,A8,"Contrary to what the Commission claims, that i...",F,NaN,37480
84,A2009_3F v Commission of the European Communities,1,prem,B1,"It should be noted, to begin with, that, when ...",L,Prec,40278
1775,R2016_DTS Distribuidora de Televisión Digital,4,prem,G1,Regarding the admissibility of the cross-appea...,L,Rule,53732
1725,R2016_DTS Distribuidora de Televisión Digital,4,prem,C1,"It should be recalled that, in paragraphs 92 a...",F,NaN,28374
1468,R2011_France Télécom SA v European Commission,5,prem,B27,The first and third parts of the second ground...,F,NaN,43133


In [ ]:
print(data['Split'].value_counts())

Split
5    702
4    535
1    483
3    448
2    367
Name: count, dtype: int64


In [ ]:
def analyze_multilabel_column(data, column_name):
    # Filter non-null values
    labeled_data = data[data[column_name].notnull()].copy()

    # Parse strings into lists
    def parse(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except:
                return [val]
        return val

    labeled_data[column_name] = labeled_data[column_name].apply(parse)

    # Count each item occurrence
    all_items = list(chain.from_iterable(labeled_data[column_name]))
    item_counts = Counter(all_items)

    # Convert to DataFrame
    item_counts_df = pd.DataFrame(item_counts.items(), columns=[column_name, 'Count']).sort_values(by='Count', ascending=False)

    # Count number of labels per example
    labeled_data[f'Num_{column_name}s'] = labeled_data[column_name].apply(len)
    multi_label_distribution = labeled_data[f'Num_{column_name}s'].value_counts().sort_index()
    multi_label_distribution_df = multi_label_distribution.reset_index().rename(columns={'index': 'Num_Labels', f'Num_{column_name}s': 'Count'})

    # Show results
    print(f"=== {column_name} Distribution ===")
    print(item_counts_df)

    print(f"\n=== Multi-label Distribution for {column_name} ===")
    print(multi_label_distribution_df)


# print results
analyze_multilabel_column(data, 'Type')
print(40 * "=")
analyze_multilabel_column(data, 'Name')
print(40 * "=")
analyze_multilabel_column(data, 'Scheme')

=== Type Distribution ===
  Type  Count
1    F   1576
0    L    906

=== Multi-label Distribution for Type ===
   Count  count
0      1   2268
1      2    107
=== Name Distribution ===
   Name  Count
0  prem   2375
1  conc    160

=== Multi-label Distribution for Name ===
   Count  count
0      1   2535
=== Scheme Distribution ===
  Scheme  Count
2   Prec    504
0   Rule    323
1   Itpr    298
3  Class     56
5    Aut     53
4  Princ     16

=== Multi-label Distribution for Scheme ===
   Count  count
0      1    615
1      2    253
2      3     43


Explaination of Scheme labels

| **Scheme** | **Meaning**               | **Typical Use in Legal Reasoning**                                                                                                                     |
| ---------- | ------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **Rule**   | **Application of a rule** | A legal norm is stated or applied to facts. <br> *E.g., “According to Article 10, the act is prohibited.”*                                             |
| **Prec**   | **Precedent**             | Refers to prior case law or decisions. <br> *E.g., “In *Smith v. Jones*, the court held that...”*                                                      |
| **Itpr**   | **Interpretation**        | Interprets laws or legal terms. <br> *E.g., “The term ‘public’ in this context means...”*                                                              |
| **Class**  | **Classification**        | Classifies a case/fact under a legal category. <br> *E.g., “This conduct falls under unfair competition.”*                                             |
| **Aut**    | **Authority**             | Appeals to legal authority beyond precedent (e.g., legal doctrine, scholars, institutions). <br> *E.g., “According to the Commission’s guidelines...”* |
| **Princ**  | **Principle**             | Refers to abstract legal principles (like justice, equality). <br> *E.g., “This violates the principle of proportionality.”*                           |

615 sentences are annotated with exactly 1 scheme

253 sentences have 2 scheme labels

43 sentences are annotated with 3 different schemes

In [ ]:
def parse_column_to_list(data, column_name):
    def parse(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except:
                return [val]
        return val
    data[column_name] = data[column_name].apply(parse)
    return data

def filter_strict_subset_rows(data, column_name, target_labels):
    """Keep only rows where the column is a list with labels that are a subset of the target set."""
    target_set = set(target_labels)

    def is_strict_subset(val):
        if not isinstance(val, list):
            return False
        return set(val).issubset(target_set)

    # Make sure column values are parsed to lists
    data = parse_column_to_list(data, column_name)
    return data[data[column_name].apply(is_strict_subset)].copy()

# Your target labels
scheme_targets = ['Aut', 'Class', 'Princ'] # Scheme
name_targets = ['conc'] # Name
type_targets = ['L'] # Type


# Filter only rows where Scheme label is combination of {Aut, Class, Princ}
filtered_scheme = filter_strict_subset_rows(data, 'Scheme', scheme_targets)

# Filter only rows where Name label is {conc}
filtered_name = filter_strict_subset_rows(data, 'Name', name_targets)

# Filter only rows where Type label is {L}
filtered_type = filter_strict_subset_rows(data, 'Type', type_targets)

label_counts_scheme = Counter(chain.from_iterable(filtered_scheme['Scheme']))
print("Filtered scheme label counts:", dict(label_counts_scheme))

label_counts_name = Counter(chain.from_iterable(filtered_name['Name']))
print("Filtered name label counts:", dict(label_counts_name))

label_counts_type = Counter(chain.from_iterable(filtered_type['Type']))
print("Filtered type label counts:", dict(label_counts_type))

Filtered scheme label counts: {'Aut': 33, 'Class': 27, 'Princ': 4}
Filtered name label counts: {'conc': 160}
Filtered type label counts: {'L': 799}


In [ ]:
# Test to see the above code works well
# Convert DataFrame rows to list of dicts
records = filtered_scheme.to_dict(orient='records')

# Save full records
with open("filtered_scheme_full.json", "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)

## Augmentation with Embedding method (WordNet + Glove)

In [ ]:
# Download the file glove.6B.zip from the Stanford NLP website
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2025-07-06 16:43:31--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-07-06 16:43:32--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.02MB/s    in 2m 39s  

2025-07-06 16:46:11 (5.18 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [ ]:
# Install the NLTK (Natural Language Toolkit) library
!pip install nltk

In [ ]:
import nltk

# Download all required resources
nltk.download('punkt_tab')                        # Tokenizer
nltk.download('averaged_perceptron_tagger_eng')   # POS tagger
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')                          # WordNet for synonyms
nltk.download('omw-1.4')                          # WordNet multilingual data (helps with lemma mapping)
nltk.download('stopwords')                        # Stop Words

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords,wordnet
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
import string
import numpy as np
import re
import random

In [ ]:
# Testing
sentence = "The judge delivered the final decision."
tokens = word_tokenize(sentence)
tags = pos_tag(tokens)

print(tags)

[('The', 'DT'), ('judge', 'NN'), ('delivered', 'VBD'), ('the', 'DT'), ('final', 'JJ'), ('decision', 'NN'), ('.', '.')]


In [ ]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{"hadn't", "we'll", 'because', 'in', 'any', 'just', 'an', 'being', 'isn', "i'll", 'whom', "he'll", 'her', 'same', "haven't", 'then', 'between', 'this', 'now', 'were', "you're", 'is', 'while', 'for', 'hasn', 'the', 'its', 'shan', 'after', 'once', 'themselves', 'there', 'again', 'through', 'me', 'have', 'under', 'm', "that'll", 'itself', 'yourselves', 'over', 'our', 'your', 'few', 'i', 'he', 'didn', 'of', 'their', "couldn't", 'a', "he's", 'ma', 'not', 'aren', 'and', 'by', 'you', 'but', 'on', 'or', 'haven', 'it', 'that', 'him', "we've", "they'll", 'doesn', 'all', 'out', 'won', 'at', 'weren', 'should', 'below', 'does', "mustn't", 'each', 'having', "it'd", "hasn't", 'll', 'too', 'they', 'so', 'did', "he'd", 'both', 'these', 'up', "you'll", 'why', 'from', 'above', 'hers', 'some', "didn't", "aren't", "they'd", 'theirs', 'himself', 'myself', "i'd", 'how', 'only', 't', 'until', 'them', 'd', 'ain', 's', "don't", 'my', 'mustn', "you've", "weren't", "i've", 'been', 'be', 're', 'hadn', 'into', "mig

In [ ]:
# Path to GloVe file
glove_path = '/content/glove.6B.100d.txt'

# Initialize the embedding dictionary
embeddings = {}

# Read the file line by line
with open(glove_path, 'r', encoding='utf8') as file:
    for line in file:
        parts = line.strip().split()
        word = parts[0]
        vector = np.array(parts[1:], dtype='float32')
        embeddings[word] = vector

In [ ]:
print(len(embeddings))

400000


In [ ]:
def cosine_similarity(vec1, vec2):
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return -1
    return np.dot(vec1, vec2) / (norm1 * norm2)

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def most_similar_word(word, candidates, embeddings):
    """
    Given a word and a list of synonym candidates,
    return the one most similar to the word using GloVe cosine similarity.
    """
    if word not in embeddings:
        return candidates[0]  # fallback

    base_vec = embeddings[word].reshape(1, -1)  # make 2D for sklearn
    best_score = -1
    best_candidate = candidates[0]

    for candidate in candidates:
        candidate_words = candidate.lower().replace('_', ' ').replace('-', ' ').split()
        vectors = [embeddings[w] for w in candidate_words if w in embeddings]

        if not vectors:
            continue  # skip if none of the words have embeddings

        candidate_vec = np.mean(vectors, axis=0).reshape(1, -1)  # 2D for sklearn
        score = cosine_similarity(base_vec, candidate_vec)[0][0]

        if score > best_score:
            best_score = score
            best_candidate = candidate

    return best_candidate.replace('_', ' ')

def synonyms(word, n_desired=5):
    """
    Find up to n_desired synonyms for the input word using WordNet,
    prioritizing those with the same POS tag.
    """
    synsets = wordnet.synsets(word)
    if not synsets:
        return [], 0

    # Get POS tag of input word (e.g., 'N', 'J', 'R', etc.)
    word_pos = pos_tag([word])[0][1][0]

    candidates = []
    priority_flags = []

    for syn in synsets:
        for lemma in syn.lemmas():
            lemma_name = lemma.name().lower()

            # Skip the original word and duplicates
            if lemma_name == word.lower() or lemma_name in candidates:
                continue

            # Prioritize matching POS types
            syn_pos = syn.pos()
            if (word_pos == 'N' and syn_pos == 'n') or \
               (word_pos == 'J' and syn_pos in ['a', 's']) or \
               (word_pos == 'R' and syn_pos == 'r'):
                priority = 1
            else:
                priority = 0

            candidates.append(lemma_name)
            priority_flags.append(priority)

    if not candidates:
        return [], 0

    # Reorder candidates: matching POS tags first
    prioritized = [w for w, p in zip(candidates, priority_flags) if p == 1]
    non_prioritized = [w for w, p in zip(candidates, priority_flags) if p == 0]
    result = prioritized + non_prioritized

    return result[:n_desired], 1

In [ ]:
# Example
word = "principle"
candidates = ["rule", "doctrine", "precept", "code_of_ethics"]

best = most_similar_word(word, candidates, embeddings)
print("Best synonym:", best)

Best synonym: doctrine


In [ ]:
# Example
words, found = synonyms("principle", 5)
print(words)

['rule', 'precept', 'rationale']


In [ ]:
all_words = []
for line in data['Text']:
    tokens = nltk.word_tokenize(line.lower())
    all_words.extend(tokens)

print(Counter(all_words).most_common(30))  # See most common words

[('the', 14050), (',', 10213), ('of', 7829), ('in', 4437), ('that', 4355), ('to', 4137), ('and', 2719), ('it', 1727), ('a', 1656), ('by', 1490), ('not', 1437), ('is', 1431), ('court', 1405), ('commission', 1370), ('as', 1361), ('be', 1267), ('general', 1107), ('paragraph', 1070), ('which', 1004), ('judgment', 984), ('(', 963), (')', 963), ('on', 960), ('appeal', 936), ('aid', 913), ('from', 828), ('under', 818), ('for', 783), ('at', 772), ('was', 671)]


## Cleaning Dataset

In [ ]:
# Define the pattern for unusual characters (non-alphanumeric, non-standard punctuation)
pattern = r'[^a-zA-Z0-9\s\.,;!?\'\"()\-]'

# Extract just the unusual characters per line
def extract_unusual(text):
    return re.findall(pattern, text)

# Apply to dataset
unusual_chars = data['Text'].apply(extract_unusual)

# Filter rows that have any unusual characters
unusual_rows = data[unusual_chars.apply(lambda x: len(x) > 0)]

# Show index + unusual characters for sample rows
for idx, chars in unusual_chars[unusual_rows.index].sample(30, random_state=1).items():
    print(f"Index {idx} → Unusual characters: {set(chars)}")

Index 1804 → Unusual characters: {'‑', ':', '/'}
Index 856 → Unusual characters: {'’'}
Index 2394 → Unusual characters: {'’'}
Index 1578 → Unusual characters: {'[', '/', ']'}
Index 1850 → Unusual characters: {'ü'}
Index 468 → Unusual characters: {'‘', '’', '‑', ':', '/'}
Index 2029 → Unusual characters: {'é'}
Index 190 → Unusual characters: {'‑', '[', '/', ']'}
Index 862 → Unusual characters: {'’'}
Index 120 → Unusual characters: {'‑', '[', '/', ']'}
Index 1705 → Unusual characters: {'‑', ':', '/'}
Index 112 → Unusual characters: {'’'}
Index 2063 → Unusual characters: {'’'}
Index 481 → Unusual characters: {'’', 'ñ'}
Index 1740 → Unusual characters: {'’'}
Index 2057 → Unusual characters: {'’'}
Index 327 → Unusual characters: {'’', '/'}
Index 1596 → Unusual characters: {'’', '–'}
Index 104 → Unusual characters: {'’'}
Index 1583 → Unusual characters: {'[', '‘', ']'}
Index 420 → Unusual characters: {'—', '’'}
Index 367 → Unusual characters: {'š'}
Index 342 → Unusual characters: {'‑', '[', 

In [ ]:
# Define the pattern: anything NOT a-z, A-Z, 0-9, whitespace, or basic punctuation
pattern = r'[^a-zA-Z0-9\s\.,;!?\'\"()-]'

# Extract all unusual characters across the dataset
all_unusual = set()

for line in data['Text']:
    chars = re.findall(pattern, line)
    all_unusual.update(chars)

# Sort and print
unusual_sorted = sorted(all_unusual)
print("Unusual characters found:", unusual_sorted)
print(f"\nTotal unique unusual characters: {len(unusual_sorted)}")

Unusual characters found: ['%', '/', ':', '>', '[', ']', '`', 'Á', 'É', 'à', 'á', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'í', 'ï', 'ñ', 'ó', 'ö', 'ú', 'ü', 'ý', 'ě', 'ő', 'š', 'ű', 'ό', '‑', '–', '—', '‘', '’', '“', '”', '…']

Total unique unusual characters: 38


In [ ]:
def cleaning(text, track_unusual=False):
    """
    Cleans a line of text by:
    - Removing HTML/XML tags
    - Lowercasing text
    - Removing numbers
    - Removing punctuation and known unusual characters
    - Normalizing whitespace
    Returns:
        cleaned_text if track_unusual is False
        (cleaned_text, set_of_unusual_characters) if track_unusual is True
    """
    original_text = text
    text = text.lower()

    # Remove tags like <ref>
    text = re.sub(r'<.*?>', '', text)

    # Remove numbers
    text = re.sub(r'\d+', ' ', text)

    # Remove known unusual characters
    for ch in unusual_sorted:
        text = text.replace(ch, ' ')

    # Remove remaining punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    if track_unusual:
        # Extract characters not in normal ranges
        unusual = set(re.findall(r'[^a-zA-Z0-9\s\.,;!?\'\"()-]', original_text))
        return text, unusual
    else:
        return text

In [ ]:
# Example
sample = "This is a <tag>sample</tag> line… with ‘quotes’ and 2024 numbers—also punctuation!"
print(cleaning(sample))

this is a sample line with quotes and numbers also punctuation


## WordNet+GloVe

In [ ]:
def augWN_GV(text, embeddings, stop_words, percent=0.6):
    """
    Apply WordNet+GloVe augmentation:
    - Replace a % of non-stopword nouns/adjectives/adverbs with WordNet synonyms,
      selected by highest cosine similarity using GloVe vectors.
    """
    # Tokenize, clean, lowercase
    tokens = nltk.word_tokenize(cleaning(text).capitalize())
    tokens = [t.lower() for t in tokens]

    # Remove stop words
    non_stop_tokens = [t for t in tokens if t not in stop_words]

    # POS tagging on non-stopwords
    tagged = nltk.pos_tag(non_stop_tokens)

    # Select replaceable tokens: adjectives, nouns, adverbs
    to_change = [t for t in tagged if t[1][0] in {'J', 'N', 'R'}]

    if not to_change:
        return text  # no eligible words

    # Determine how many words to replace
    n_replace = max(1, round(percent * len(to_change)))

    # For replacement tracking
    modified_text = text.lower()
    capitalized = text[0].isupper()
    changed = 0
    index = -1

    # Try replacements until desired count is met
    while changed < n_replace:
        index += 1
        if index >= len(to_change):
            break  # no more candidates

        word = to_change[index][0]

        # Get synonyms
        candidates, found = synonyms(word, 10)
        if not found or not candidates:
            continue

        # Pick the best synonym based on GloVe
        replacement = most_similar_word(word, candidates, embeddings)
        if replacement == word:
            continue  # skip if no real change

        # Replace only one occurrence surrounded by non-alphabetic
        pattern = rf'(?<![a-zA-Z#]){re.escape(word)}(?![a-zA-Z#])'
        modified_text = re.sub(pattern, f'#{replacement}#', modified_text, 1)
        changed += 1

    # Remove '#' markers and restore capitalization
    final = modified_text.replace('#', '')
    if capitalized:
        final = final.capitalize()
    return final

## Renew the Dataset with WordNet+Glove method

In [ ]:
# Copy original data and mark
data_new = data.copy()
data_new['Augmentation_Flag'] = 'original'

# Apply the filter
scheme_targets = ['Aut', 'Class', 'Princ']
filtered_scheme = filter_strict_subset_rows(data_new, 'Scheme', scheme_targets)

name_target = ['conc']
filtered_name = filter_strict_subset_rows(data_new, 'Name', name_targets)

# View class distribution after filtering
label_counts_scheme = Counter(chain.from_iterable(filtered_scheme['Scheme']))
print("Filtered scheme label counts:", dict(label_counts_scheme))

label_counts_name = Counter(chain.from_iterable(filtered_name['Name']))
print("Filtered name label counts:", dict(label_counts_name))

# Function to clean and augment
def clean_and_augment(df):
    df = df.copy()
    df['Clean_Text'] = df['Text'].apply(cleaning)
    df['Augmented_Text'] = df['Clean_Text'].apply(lambda x: augWN_GV(x, embeddings, stop_words))
    return df

filtered_scheme = clean_and_augment(filtered_scheme)
filtered_name = clean_and_augment(filtered_name)

# Function to save augmented data
def save_augmented_texts(df, label_column, output_path, label_name):
    with open(output_path, 'w', encoding='utf-8') as f:
        for label, orig, aug in zip(df[label_column], df['Clean_Text'], df['Augmented_Text']):
            # Handle list labels like ['Aut'] or ['conc']
            label_str = ", ".join(label) if isinstance(label, list) else str(label)
            f.write(f"{label_name.upper()}: {label_str}\n")
            f.write(f"ORIGINAL: {orig}\n")
            f.write(f"AUGMENTED: {aug}\n\n")
    print(f"{label_name.capitalize()} augmented data saved to: {output_path}")

# Save scheme and name files
save_augmented_texts(filtered_scheme, 'Scheme', '/content/augmented_scheme.txt', 'scheme')
save_augmented_texts(filtered_name, 'Name', '/content/augmented_name.txt', 'name')

# General augmentation builder
def build_augmented_rows(df, cols_to_nullify):
    augmented = df.copy()
    augmented['Text'] = augmented['Augmented_Text']
    augmented['Augmentation_Flag'] = 'augmented'
    for col in cols_to_nullify:
        if col in augmented.columns:
            augmented[col] = None
    return augmented

# Create augmented data
augmented_scheme_df = build_augmented_rows(filtered_scheme, cols_to_nullify=['Name', 'Type'])
augmented_name_df = build_augmented_rows(filtered_name, cols_to_nullify=['Scheme', 'Type'])

# Combine original and augmented data
augmented_df = pd.concat([augmented_scheme_df, augmented_name_df], ignore_index=True)
final_data = pd.concat([data_new, augmented_df], ignore_index=True)

# Drop temp columns
final_data.drop(columns=['Clean_Text', 'Augmented_Text'], inplace=True, errors='ignore')

# Save
final_data.to_pickle("/content/drive/MyDrive/Colab Notebooks/final_augmented_data.pkl")

# Sample output
print(final_data[['Text', 'Scheme', 'Augmentation_Flag']].sample(10))

Filtered scheme label counts: {'Aut': 33, 'Class': 27, 'Princ': 4}
Filtered name label counts: {'conc': 160}
Scheme augmented data saved to: /content/augmented_scheme.txt
Name augmented data saved to: /content/augmented_name.txt
                                                   Text               Scheme  \
1667  In addition, in order to determine whether the...                  NaN   
121   In the present case, as the Court of First Ins...  [Class, Prec, Rule]   
287   addition, the General Court relied on other gr...                  NaN   
240   the consequence of this would be that national...               [Itpr]   
815   As is apparent from the case-law referred to i...                  NaN   
2177  It is settled case-law that the statement of r...         [Prec, Rule]   
989   In the second part of the first ground of appe...                  NaN   
1991  Accordingly, the second part of the third grou...                  NaN   
1998  The Commission considered, however, that thos

In [ ]:
final_data = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/final_augmented_data.pkl")
final_data.sample(20)

,Document,Split,Name,Id,Text,Type,Scheme,Start,Augmentation_Flag
2530,R2021_World Duty Free v,5,[prem],H7,Such a question is a matter for dialogue betwe...,F,NaN,86942,original
1748,R2016_DTS Distribuidora de Televisión Digital,4,[prem],D21,"However, in the present case, the amount of th...",F,NaN,42039,original
2542,A2017_European Commission v TV2_Danmark A_S,5,None,A2bis,for the first time there must be intervention ...,None,[Class],18142,augmented
1075,R2002_Associação dos Refinadores de Açúcar Por...,1,[prem],D11,"Rather, in that decision, the Commission requi...",F,NaN,53205,original
238,A2011_European Commission (C-106_09 P) and Kin...,2,[prem],B9,"However, contrary to the General Court’s reaso...","[F, L]",[Itpr],49562,original
2665,R2004_Italian Republic v Commission of the Eur...,1,[conc],A12,consequently the military commission s crossin...,None,None,38737,augmented
1556,R2012_European Commission v Électricité de Fra...,5,[prem],A40,It has already been noted in paragraphs 83 to ...,L,[Itpr],54275,original
401,A2016_European Commission v Aer Lingus Ltd and...,3,[prem],A5,"It is the Court’s established case-law that, w...",L,"[Prec, Rule]",27740,original
1004,R2000_French Republic v Ladbroke Racing Ltd an...,1,[prem],C9,The judgment in Case T-358/94 Air France v Com...,L,"[Prec, Rule]",37631,original
1678,R2015_European Commission v MOL Magyar Olaj- é...,3,[prem],B7,"In any event, it must be stated that, as MOL c...",F,NaN,41243,original


In [ ]:
print(final_data['Augmentation_Flag'].value_counts())

Augmentation_Flag
original     2535
augmented     224
Name: count, dtype: int64


In [ ]:
# print results
analyze_multilabel_column(final_data, 'Type')
print(40 * "=")
analyze_multilabel_column(final_data, 'Name')
print(40 * "=")
analyze_multilabel_column(final_data, 'Scheme')

=== Type Distribution ===
  Type  Count
1    F   1576
0    L    906

=== Multi-label Distribution for Type ===
   Count  count
0      1   2268
1      2    107
=== Name Distribution ===
   Name  Count
0  prem   2375
1  conc    320

=== Multi-label Distribution for Name ===
   Count  count
0      1   2695
=== Scheme Distribution ===
  Scheme  Count
2   Prec    504
0   Rule    323
1   Itpr    298
5    Aut     86
3  Class     83
4  Princ     20

=== Multi-label Distribution for Scheme ===
   Count  count
0      1    679
1      2    253
2      3     43


## Augmentation with Agent based method

In [ ]:
!pip install camel-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.5
    Uninstalling psutil-5.9.5:
      Successfully uninstalled psutil-5.9.5
  Attempting uninstall: pillow
    Found existing installation: pillow 11.2.1
    Uninstalling pillow-11.2.1:
      Successfully uninstalled pillow-11.2.1
  Attempting uninstall: docstring-parser
    Found existing in

In [ ]:
import os
import dotenv
import pandas as pd
import sys
from collections import Counter, ChainMap
from itertools import chain
import pandas as pd

from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.messages.base import BaseMessage

# Append directory to sys.path
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Augmentation')

# Check if config.py is in the path
assert 'config.py' in os.listdir('/content/drive/MyDrive/Colab Notebooks/Augmentation'), "config.py not found!"

# Now import
from config import OPENROUTER_API_KEY

In [ ]:
# Copy original data and mark
data_new2 = data.copy()
data_new2['Augmentation_Flag'] = 'original'

# Apply the filter
scheme_targets = ['Aut', 'Class', 'Princ']
name_targets = ['conc']                      # <- fixed typo

filtered_scheme = filter_strict_subset_rows(data_new2, 'Scheme', scheme_targets)
filtered_name = filter_strict_subset_rows(data_new2, 'Name',   name_targets)

# Quick label distribution check
print("Filtered scheme label counts:", dict(Counter(chain.from_iterable(filtered_scheme['Scheme']))))
print("Filtered name label counts:", dict(Counter(chain.from_iterable(filtered_name['Name']))))

# Agent Setup
AGENT_SYSTEM_PROMPT = """You are a generation new text agent.
You will be given a context about legal text.
Your goal is to analyze context and try to augment new text while retaining the meaning.
Try to answer with a text of similar size.
"""

model_platform = ModelPlatformType.OPENROUTER
worker_model_type = "google/gemini-2.5-flash-preview"
worker_model_config = {"max_tokens": 4096}

def create_agent_model(platform, model_type, key, config):
    return ModelFactory.create(
        model_platform=platform,
        model_type=model_type,
        api_key=key,
        model_config_dict=config,
    )

class SingleAgentSolver:
    def __init__(self):
        self.model_platform = model_platform
        self.api_key = OPENROUTER_API_KEY
        self.augmentation_agent = self.create_agent()

    def create_agent(self):
        model = create_agent_model(
            self.model_platform,
            worker_model_type,
            self.api_key,
            worker_model_config,
        )
        system_message = BaseMessage.make_assistant_message(
            role_name="Augmentation agent",
            content=AGENT_SYSTEM_PROMPT,
        )
        return ChatAgent(system_message=system_message, model=model, tools=[])

    def reset(self):
        """Reset the thinking toolkit and rebuild agents for a clean slate."""
        self.augmentation_agent = self.create_agent()

    def augment_text(self, text: str) -> str:
        """Return a single augmented version (fallback: return original)."""
        try:
            user_msg = BaseMessage.make_user_message("User", text)
            self.augmentation_agent.reset()
            result = self.augmentation_agent.step(user_msg)
            return result.msg.content.strip()
        except Exception as e:
            # Logging
            print(f"[Augmentation failed] {text[:60]}…  -> {e}")
            return text

# Apply the agent
solver = SingleAgentSolver()

def prepare_for_agent(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['Clean_Text'] = out['Text'].apply(cleaning)
    out['Augmented_Text'] = out['Clean_Text'].apply(solver.augment_text)
    return out

filtered_scheme = prepare_for_agent(filtered_scheme)
filtered_name   = prepare_for_agent(filtered_name)

# Function to save augmented data
def save_augmented_texts(df, label_col, path, label_name):
    with open(path, 'w', encoding='utf-8') as f:
        for label, orig, aug in zip(df[label_col], df['Clean_Text'], df['Augmented_Text']):
            label_str = ", ".join(label) if isinstance(label, list) else str(label)
            f.write(f"{label_name.upper()}: {label_str}\n")
            f.write(f"ORIGINAL : {orig}\n")
            f.write(f"AUGMENTED: {aug}\n\n")
    print(f"{label_name.capitalize()} augmented data written to {path}")

save_augmented_texts(filtered_scheme, 'Scheme', '/content/augmented_scheme_agent.txt', 'scheme')
save_augmented_texts(filtered_name,   'Name', '/content/augmented_name_agent.txt',   'name')

# General augmentation builder
def build_augmented_rows(df, aug_col='Augmented_Text', nullify_cols=None):
    """Return a df where Text is replaced by `aug_col`, flag set to 'augmented',
    and `nullify_cols` (if provided) are overwritten with None."""
    if nullify_cols is None:
        nullify_cols = []
    out = df.copy()
    out['Text']              = out[aug_col]
    out['Augmentation_Flag'] = 'augmented'
    for col in nullify_cols:
        if col in out.columns:
            out[col] = None
    return out

augmented_scheme_df = build_augmented_rows(filtered_scheme, nullify_cols=['Name', 'Type'])
augmented_name_df   = build_augmented_rows(filtered_name, nullify_cols=['Scheme', 'Type'])

# Combine original and augmented data
augmented_df = pd.concat([augmented_scheme_df, augmented_name_df], ignore_index=True)

final_data_agent = pd.concat([data_new2, augmented_df], ignore_index=True)

final_data_agent.drop(columns=['Clean_Text', 'Augmented_Text'], inplace=True, errors='ignore')

final_data_agent.to_pickle("/content/drive/MyDrive/Colab Notebooks/final_augmented_data_agent.pkl")

print(final_data_agent[['Text', 'Scheme', 'Augmentation_Flag']].sample(10))

Filtered scheme label counts: {'Aut': 33, 'Class': 27, 'Princ': 4}
Filtered name label counts: {'conc': 160}
Scheme augmented data written to /content/augmented_scheme_agent.txt
Name augmented data written to /content/augmented_name_agent.txt
                                                   Text         Scheme  \
1195  In paragraph 92 of the contested judgment, the...            NaN   
1671  The first part of the single ground of appeal ...            NaN   
2511  Suffice it to note in that regard that, as is ...            NaN   
791   The examination of the selectivity condition t...  [Class, Prec]   
113   The appellant, which is only one of the many t...            NaN   
1755  Thus, those liable to pay a charge cannot rely...   [Itpr, Prec]   
781   First, there must be an intervention by the St...        [Class]   
1424  As regards the present case, it should be note...            NaN   
2738  Consequently, the fourth ground of appeal is t...           None   
944   The share o

In [ ]:
final_data_agent = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/final_augmented_data_agent.pkl")
final_data_agent.sample(20)

,Document,Split,Name,Id,Text,Type,Scheme,Start,Augmentation_Flag
1515,R2012_European Commission v Électricité de Fra...,5,[prem],A2,"secondly, it confused the roles of the State a...",F,NaN,40645,original
114,A2009_3F v Commission of the European Communities,1,[prem],D11,It was not directly involved in the adoption b...,F,NaN,55874,original
935,R1997_Tiercé Ladbroke SA v Commission of the E...,1,[prem],A7,"According to the Court of First Instance, the ...",F,NaN,17226,original
256,A2011_European Commission (C-106_09 P) and Kin...,2,[prem],B27,"Thus, the fact that offshore companies, which ...",F,NaN,55360,original
2526,R2021_World Duty Free v,5,[prem],H4,As has been pointed out in paragraph 146 above...,"[F, L]",[Itpr],85891,original
305,A2012_BNP Paribas and Banca Nazionale del Lavo...,2,[prem],C16,"However, the General Court did not find that t...",F,NaN,55237,original
1334,R2010_AceaElectrabel Produzione SpA v European...,2,[prem],B20,In the light of the case‑law cited at paragrap...,F,NaN,28394,original
103,A2009_3F v Commission of the European Communities,1,[prem],D1,It should be recalled that in Van der Kooy the...,L,[Prec],52071,original
2108,R2017_European Commission v Frucona Košice a,2,[conc],B6,"In those circumstances, the first ground of ap...",NaN,NaN,27426,original
1009,R2000_French Republic v Ladbroke Racing Ltd an...,1,[prem],D3,"Furthermore, the Court of First Instance could...",F,NaN,39209,original


In [ ]:
print(final_data_agent['Augmentation_Flag'].value_counts())

Augmentation_Flag
original     2535
augmented     224
Name: count, dtype: int64


In [ ]:
# print results
analyze_multilabel_column(final_data_agent, 'Type')
print(40 * "=")
analyze_multilabel_column(final_data_agent, 'Name')
print(40 * "=")
analyze_multilabel_column(final_data_agent, 'Scheme')

=== Type Distribution ===
  Type  Count
1    F   1576
0    L    906

=== Multi-label Distribution for Type ===
   Count  count
0      1   2268
1      2    107
=== Name Distribution ===
   Name  Count
0  prem   2375
1  conc    320

=== Multi-label Distribution for Name ===
   Count  count
0      1   2695
=== Scheme Distribution ===
  Scheme  Count
2   Prec    504
0   Rule    323
1   Itpr    298
5    Aut     86
3  Class     83
4  Princ     20

=== Multi-label Distribution for Scheme ===
   Count  count
0      1    679
1      2    253
2      3     43


In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Paths to the files
llm_path = "/content/augmented_scheme_agent.txt"
embed_path = "/content/sample_data/augmented_scheme (4).txt"

# Function to parse augmented files
def parse_augmented_file(filepath):
    originals, augmenteds = [], []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip().startswith("ORIGINAL") and ':' in line:
                text = line.split(":", 1)[1].strip()
                originals.append(text)
            elif line.strip().startswith("AUGMENTED") and ':' in line:
                text = line.split(":", 1)[1].strip()
                augmenteds.append(text)
    return originals, augmenteds

# Compute cosine similarities between sentence pairs
def compute_cosine_similarities(originals, augmenteds):
    if len(originals) != len(augmenteds):
        raise ValueError(f"Mismatch: {len(originals)} originals vs {len(augmenteds)} augmenteds")
    emb1 = model.encode(originals, convert_to_tensor=True)
    emb2 = model.encode(augmenteds, convert_to_tensor=True)
    return util.cos_sim(emb1, emb2).diagonal().cpu().numpy()

# Parse both files
orig_llm, aug_llm = parse_augmented_file(llm_path)
orig_embed, aug_embed = parse_augmented_file(embed_path)

# Log the number of parsed entries
print(f"Parsed {len(orig_llm)} LLM pairs and {len(orig_embed)} embedding pairs.")

# Compute similarities
sim_llm = compute_cosine_similarities(orig_llm, aug_llm)
sim_embed = compute_cosine_similarities(orig_embed, aug_embed)

# Report
print("LLM Avg Similarity:       ", round(np.mean(sim_llm), 4))
print("Embedding Avg Similarity: ", round(np.mean(sim_embed), 4))

Parsed 64 LLM pairs and 64 embedding pairs.
LLM Avg Similarity:        0.8321
Embedding Avg Similarity:  0.8027


In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Paths to the files
llm_path = "/content/sample_data/augmented_name_agent.txt"
embed_path = "/content/sample_data/augmented_name (3).txt"

# Function to parse augmented files
def parse_augmented_file(filepath):
    originals, augmenteds = [], []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip().startswith("ORIGINAL") and ':' in line:
                text = line.split(":", 1)[1].strip()
                originals.append(text)
            elif line.strip().startswith("AUGMENTED") and ':' in line:
                text = line.split(":", 1)[1].strip()
                augmenteds.append(text)
    return originals, augmenteds

# Compute cosine similarities between sentence pairs
def compute_cosine_similarities(originals, augmenteds):
    if len(originals) != len(augmenteds):
        raise ValueError(f"Mismatch: {len(originals)} originals vs {len(augmenteds)} augmenteds")
    emb1 = model.encode(originals, convert_to_tensor=True)
    emb2 = model.encode(augmenteds, convert_to_tensor=True)
    return util.cos_sim(emb1, emb2).diagonal().cpu().numpy()

# Parse both files
orig_llm, aug_llm = parse_augmented_file(llm_path)
orig_embed, aug_embed = parse_augmented_file(embed_path)

# Log the number of parsed entries
print(f"Parsed {len(orig_llm)} LLM pairs and {len(orig_embed)} embedding pairs.")

# Compute similarities
sim_llm = compute_cosine_similarities(orig_llm, aug_llm)
sim_embed = compute_cosine_similarities(orig_embed, aug_embed)

# Report
print("LLM Avg Similarity:       ", round(np.mean(sim_llm), 4))
print("Embedding Avg Similarity: ", round(np.mean(sim_embed), 4))

Parsed 160 LLM pairs and 160 embedding pairs.
LLM Avg Similarity:        0.7732
Embedding Avg Similarity:  0.8107
